# TV Script Generation
In this project, you'll generate your own [Simpsons](https://en.wikipedia.org/wiki/The_Simpsons) TV scripts using RNNs.  You'll be using part of the [Simpsons dataset](https://www.kaggle.com/wcukierski/the-simpsons-by-the-data) of scripts from 27 seasons.  The Neural Network you'll build will generate a new TV script for a scene at [Moe's Tavern](https://simpsonswiki.com/wiki/Moe's_Tavern).
## Get the Data
The data is already provided for you.  You'll be using a subset of the original dataset.  It consists of only the scenes in Moe's Tavern.  This doesn't include other versions of the tavern, like "Moe's Cavern", "Flaming Moe's", "Uncle Moe's Family Feed-Bag", etc..

In [1]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import helper

data_dir = './data/simpsons/moes_tavern_lines.txt'
text = helper.load_data(data_dir)
# Ignore notice, since we don't use it for analysing the data
text = text[81:]

In [2]:
text[:100]

"\nMoe_Szyslak: (INTO PHONE) Moe's Tavern. Where the elite meet to drink.\nBart_Simpson: Eh, yeah, hell"

## Explore the Data
Play around with `view_sentence_range` to view different parts of the data.

In [3]:
view_sentence_range = (0, 10)

"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import numpy as np

print('Dataset Stats')
print('Roughly the number of unique words: {}'.format(len({word: None for word in text.split()})))
scenes = text.split('\n\n')
print('Number of scenes: {}'.format(len(scenes)))
sentence_count_scene = [scene.count('\n') for scene in scenes]
print('Average number of sentences in each scene: {}'.format(np.average(sentence_count_scene)))

sentences = [sentence for scene in scenes for sentence in scene.split('\n')]
print('Number of lines: {}'.format(len(sentences)))
word_count_sentence = [len(sentence.split()) for sentence in sentences]
print('Average number of words in each line: {}'.format(np.average(word_count_sentence)))

print()
print('The sentences {} to {}:'.format(*view_sentence_range))
print('\n'.join(text.split('\n')[view_sentence_range[0]:view_sentence_range[1]]))

Dataset Stats
Roughly the number of unique words: 11492
Number of scenes: 262
Average number of sentences in each scene: 15.251908396946565
Number of lines: 4258
Average number of words in each line: 11.50164396430249

The sentences 0 to 10:

Moe_Szyslak: (INTO PHONE) Moe's Tavern. Where the elite meet to drink.
Bart_Simpson: Eh, yeah, hello, is Mike there? Last name, Rotch.
Moe_Szyslak: (INTO PHONE) Hold on, I'll check. (TO BARFLIES) Mike Rotch. Mike Rotch. Hey, has anybody seen Mike Rotch, lately?
Moe_Szyslak: (INTO PHONE) Listen you little puke. One of these days I'm gonna catch you, and I'm gonna carve my name on your back with an ice pick.
Moe_Szyslak: What's the matter Homer? You're not your normal effervescent self.
Homer_Simpson: I got my problems, Moe. Give me another one.
Moe_Szyslak: Homer, hey, you should not drink to forget your problems.
Barney_Gumble: Yeah, you should only drink to enhance your social skills.



## Implement Preprocessing Functions
The first thing to do to any dataset is preprocessing.  Implement the following preprocessing functions below:
- Lookup Table
- Tokenize Punctuation

### Lookup Table
To create a word embedding, you first need to transform the words to ids.  In this function, create two dictionaries:
- Dictionary to go from the words to an id, we'll call `vocab_to_int`
- Dictionary to go from the id to word, we'll call `int_to_vocab`

Return these dictionaries in the following tuple `(vocab_to_int, int_to_vocab)`

In [4]:
import numpy as np
import problem_unittests as tests

def create_lookup_tables(text):
    """
    Create lookup tables for vocabulary
    :param text: The text of tv scripts split into words
    :return: A tuple of dicts (vocab_to_int, int_to_vocab)
    """
    vocab = set(text)
    vocab_to_int = {word:ii for ii,word in enumerate(vocab,1)}
    int_to_vocab = {vocab_to_int.get(key):key for key in vocab_to_int.keys()}
    return (vocab_to_int,int_to_vocab)


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_create_lookup_tables(create_lookup_tables)

Tests Passed


### Tokenize Punctuation
We'll be splitting the script into a word array using spaces as delimiters.  However, punctuations like periods and exclamation marks make it hard for the neural network to distinguish between the word "bye" and "bye!".

Implement the function `token_lookup` to return a dict that will be used to tokenize symbols like "!" into "||Exclamation_Mark||".  Create a dictionary for the following symbols where the symbol is the key and value is the token:
- Period ( . )
- Comma ( , )
- Quotation Mark ( " )
- Semicolon ( ; )
- Exclamation mark ( ! )
- Question mark ( ? )
- Left Parentheses ( ( )
- Right Parentheses ( ) )
- Dash ( -- )
- Return ( \n )

This dictionary will be used to token the symbols and add the delimiter (space) around it.  This separates the symbols as it's own word, making it easier for the neural network to predict on the next word. Make sure you don't use a token that could be confused as a word. Instead of using the token "dash", try using something like "||dash||".

In [10]:
def token_lookup():
    """
    Generate a dict to turn punctuation into a token.
    :return: Tokenize dictionary where the key is the punctuation and the value is the token
    """
    dict_ = {',':'||Comma||','.':'||Period||','"':'||Quotation_Mark||',';':'||Semocolon||','!':'||Exclamation_Mark||',
             '?':'||Question_mark||','(':'||Left_Parentheses||',')':'||Right_Parentheses||','--':'||Dash||','\n':'||Return||'}
    return dict_

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_tokenize(token_lookup)

Tests Passed


## Preprocess all the data and save it
Running the code cell below will preprocess all the data and save it to file.

In [1]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
# Preprocess Training, Validation, and Testing Data
helper.preprocess_and_save_data(data_dir, token_lookup, create_lookup_tables)

NameError: name 'helper' is not defined

# Check Point
This is your first checkpoint. If you ever decide to come back to this notebook or have to restart the notebook, you can start from here. The preprocessed data has been saved to disk.

In [6]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import helper
import numpy as np
import problem_unittests as tests

int_text, vocab_to_int, int_to_vocab, token_dict = helper.load_preprocess()

In [7]:
for text in int_text:
    print(int_to_vocab.get(text))

||return||
moe_szyslak:
||left_parentheses||
into
phone
||right_parentheses||
moe's
tavern
||period||
where
the
elite
meet
to
drink
||period||
||return||
bart_simpson:
eh
||comma||
yeah
||comma||
hello
||comma||
is
mike
there
||question_mark||
last
name
||comma||
rotch
||period||
||return||
moe_szyslak:
||left_parentheses||
into
phone
||right_parentheses||
hold
on
||comma||
i'll
check
||period||
||left_parentheses||
to
barflies
||right_parentheses||
mike
rotch
||period||
mike
rotch
||period||
hey
||comma||
has
anybody
seen
mike
rotch
||comma||
lately
||question_mark||
||return||
moe_szyslak:
||left_parentheses||
into
phone
||right_parentheses||
listen
you
little
puke
||period||
one
of
these
days
i'm
gonna
catch
you
||comma||
and
i'm
gonna
carve
my
name
on
your
back
with
an
ice
pick
||period||
||return||
moe_szyslak:
what's
the
matter
homer
||question_mark||
you're
not
your
normal
effervescent
self
||period||
||return||
homer_simpson:
i
got
my
problems
||comma||
moe
||period||
give
me
a

a
||quotation_mark||
flaming
moe
||quotation_mark||
||exclamation_mark||
that's
right
||comma||
a
||quotation_mark||
flaming
moe
||quotation_mark||
||period||
my
name
is
moe
||comma||
and
i
invented
it
||period||
that's
why
it's
called
a
flaming
moe
||period||
what
||question_mark||
what
are
you
lookin'
at
||comma||
homer
||question_mark||
it's
a
flaming
moe
i'm
moe
||period||
||return||
barney_gumble:
hey
||comma||
what's
this
||question_mark||
||return||
moe_szyslak:
a
sneeze
guard
||period||
||return||
barney_gumble:
wow
||comma||
it
really
works
||period||
||return||
harv:
||left_parentheses||
chuckling
||right_parentheses||
i'll
be
back
||period||
||return||
homer_simpson:
moe
||comma||
i
haven't
seen
the
place
this
crowded
since
the
government
cracked
down
on
you
for
accepting
food
stamps
||period||
do
you
think
my
drink
had
something
to
do
with
it
||question_mark||
||return||
moe_szyslak:
who
can
say
||question_mark||
it's
probably
a
combination
of
things
||period||
||return||
p

sarcastic
laugh
||right_parentheses||
okay
||comma||
ya
lucky
moron
||period||
||return||
moe_szyslak:
here
you
go
||comma||
homer
||period||
a
hundred
and
thirty-five
dollars
||period||
||return||
homer_simpson:
||left_parentheses||
sagely
||right_parentheses||
i
used
to
hate
the
smell
of
your
sweaty
feet
||period||
||left_parentheses||
sniffs
deeply
||right_parentheses||
now
it's
the
smell
of
victory
||period||
||return||
moe_szyslak:
aw
||comma||
shut
up
||period||
||return||
moe_szyslak:
hundred
and
ten
||comma||
a
hundred
and
twenty
||period||
||period||
||period||
you
lucky
son-of-a
||period||
||period||
||period||
||return||
barney_gumble:
hey
homer
||comma||
you
wanna
go
bowling
next
sunday
||question_mark||
||return||
homer_simpson:
barney
||comma||
are
you
nuts
||question_mark||
that's
the
super
bowl
||period||
how
about
the
sunday
after
that
||question_mark||
||return||
barney_gumble:
well
||comma||
my
ma's
coming
in
from
norway
||comma||
but
||period||
||period||
||period||

||period||
||return||
barney_gumble:
wow
||comma||
moe
||comma||
you
didn't
even
give
a
beer
to
those
freed
iranian
hostages
||period||
||return||
teenage_barney:
i
don't
know
||period||
the
sat's
are
tomorrow
||period||
||return||
moe_szyslak:
ah
||comma||
they
shouldn't
have
been
there
in
the
first
place
||period||
but
homer
is
a
real
hero
||period||
||return||
barney_gumble:
i
wish
i
was
a
hero
||period||
||return||
homer_simpson:
well
wishing
won't
make
it
so
||period||
you've
got
to
pull
up
your
diaper
||comma||
get
out
there
||comma||
and
be
the
best
damn
barney
you
can
be
||period||
||return||
barney_gumble:
here
i
come
world
||exclamation_mark||
||return||
homer_simpson:
how
do
you
think
he'll
do
||comma||
moe
||question_mark||
||return||
moe_szyslak:
||left_parentheses||
proudly
||right_parentheses||
i
think
he'll
do
just
fine
||period||
||return||
moe_szyslak:
||left_parentheses||
to
barney
||right_parentheses||
linda
ronstadt
||question_mark||
how'd
you
get
her
||question_ma

marge_simpson:
i'm
awfully
sorry
||period||
||return||
jacques:
entirely
my
fault
||period||
||left_parentheses||
pause
||right_parentheses||
it
is
nice
to
meet
you
||period||
||period||
||period||
||left_parentheses||
looks
at
her
ball
||right_parentheses||
||period||
||period||
||period||
homer
||period||
||return||
marge_simpson:
||left_parentheses||
flustered
||right_parentheses||
oh
||comma||
no
||comma||
no
||period||
homer
is
my
||period||
||period||
||period||
ball's
name
||period||
i'm
marge
||period||
||return||
jacques:
your
fingers
are
so
slender
||comma||
so
feminine
||period||
they
are
far
too
tapered
for
the
ball
you
are
using
||period||
you
need
something
lighter
||comma||
more
delicate
||period||
here
||comma||
use
my
ball
||period||
||return||
marge_simpson:
no
||period||
||period||
||period||
no
||comma||
thank
you
||comma||
mr
||period||
||period||
||period||
uh
||period||
||period||
||period||
||left_parentheses||
looks
at
the
name
on
his
ball
||right_parentheses||

...

were
what
was
left
of
my
youth
||comma||
moe
||period||
||return||
moe_szyslak:
hey
||comma||
come
on
||comma||
there's
sexy
bald
like
||period||
||period||
||period||
uh
||period||
||period||
||period||
babar
||comma||
king
of
the
elephants
||period||
i
read
his
books
as
a
kid
||period||
||left_parentheses||
confidential
||right_parentheses||
he
married
his
cousin
||comma||
celeste
||period||
||left_parentheses||
beat
||right_parentheses||
that
was
my
takeaway
||period||
||return||
homer_simpson:
those
royal
elephants
have
trainers
to
keep
'em
in
shape
||period||
average
schmoe
like
me
||comma||
forget
it
||period||
||return||
moe_szyslak:
well
||comma||
let
me
see
how
bald
you
are
||period||
||return||
moe_szyslak:
||left_parentheses||
can't-believe-how-bald-he-is
noise
||right_parentheses||
dear
lord
||period||
here
||period||
you
need
this
more
than
i
do
||period||
||return||
moe_szyslak:
just
what
exactly
is
this
good
for
||question_mark||
||return||
||return||
||return||
homer_si

nfl_narrator:
||period||
||period||
||period||
despite
her
breathtaking
ignorance
and
a
powerful
dislike
of
the
sport
||comma||
a
rookie
housewife
had
been
defeated
by
a
savvy
veteran
||period||
||period||
||period||
||return||
||return||
||return||
marge_simpson:
||left_parentheses||
sadly
||right_parentheses||
now
i
know
why
homie
comes
here
so
much
||period||
no
matter
how
sad
you
are
inside
||comma||
what
you
see
looks
worse
||period||
||return||
moe_szyslak:
yeah
||period||
it
was
really
hard
for
me
to
make
this
place
look
old
but
not
in
any
way
comfortable
||period||
now
maybe
some
tv'll
cheer
ya
up
||period||
||return||
marge_simpson:
||left_parentheses||
a
little
tipsy
||right_parentheses||
yesterday's
whatsit
||question_mark||
why
does
everyone
go
to
them
||question_mark||
||return||
lenny_leonard:
it's
express
||comma||
marge
||period||
||return||
carl_carlson:
yeah
with
our
busy
lifestyle
||comma||
we
gotta
have
express
||period||
||return||
moe_szyslak:
uh
||comma||
listen


low
baritone
voice
||right_parentheses||
you're
not
alone
anymore
||period||
||return||
moe_szyslak:
||left_parentheses||
terrified
||right_parentheses||
what
the
hell
was
that
||question_mark||
||exclamation_mark||
||return||
smile:
||left_parentheses||
low
baritone
voice
||right_parentheses||
oh
||comma||
you'll
find
out
||period||
||return||
||return||
||return||
moe_szyslak:
really
||question_mark||
tell
me
something:
i've
heard
that
queen
elizabeth
in
person
is
actually
not
that
funny
||period||
||return||
kemi:
i
am
from
africa
||period||
||return||
moe_szyslak:
wow
||comma||
africa
||period||
i
had
a
good
friend
who
really
wanted
to
go
there
||period||
so
||comma||
uh
||comma||
you
speak
english
there
||question_mark||
||return||
kemi:
i
speak
five
languages
||period||
||return||
moe_szyslak:
||left_parentheses||
snorts
||right_parentheses||
nobody
does
||period||
||return||
kemi:
||left_parentheses||
portuguese
||right_parentheses||
eu
nã£o
quero
dizer
para
mostrar
||left_paren

||period||
||return||
homer_simpson:
you
know
||comma||
moe
||comma||
my
mom
once
said
something
that
really
stuck
with
me
||period||
she
said
||comma||
||quotation_mark||
homer
||comma||
you're
a
big
disappointment
||comma||
||quotation_mark||
and
||comma||
god
bless
her
soul
||comma||
she
was
really
onto
something
||period||
||return||
barney_gumble:
don't
blame
yourself
||comma||
homer
||period||
you
got
dealt
a
bad
hand
||period||
you
got
crummy
little
kids
that
nobody
can
control
||period||
||return||
homer_simpson:
you
can't
talk
that
way
about
my
kids
||period||
or
||comma||
at
least
two
of
them
||period||
||return||
barney_gumble:
why
||question_mark||
you
got
two
i
haven't
met
||question_mark||
||return||
homer_simpson:
why
||comma||
you
||period||
||period||
||period||
here's
five
you
haven't
met
||period||
||return||
boxing_announcer:
and
a
tremendous
right
||exclamation_mark||
that's
just
got
to
hurt
||exclamation_mark||
ladies
and
gentlemen
||comma||
this
fight
is
over
||e

alright
||comma||
gotta
look
the
wife
straight
in
the
eyes
and
tell
her
||period||
||return||
homer_simpson:
oh
||comma||
who
am
i
kidding
||question_mark||
i
love
you
more
than
ever
||period||
||return||
marge_simpson:
i
love
you
too
||exclamation_mark||
||return||
homer_simpson:
sorry
to
scare
you
like
that
||comma||
babe
||period||
||return||
moe_szyslak:
okay
||comma||
everybody
||exclamation_mark||
for
the
next
fifteen
minutes
||comma||
one
third
off
on
every
pitcher
||period||
||period||
||period||
one
per
customer
||period||
||period||
||period||
domestic
beer
only
||period||
||period||
||period||
hey
||comma||
no
sharing
||exclamation_mark||
||return||
||return||
||return||
homer_simpson:
i
gotta
call
marge
||period||
||return||
barney_gumble:
ha
||exclamation_mark||
||return||
moe_szyslak:
||left_parentheses||
into
phone
||right_parentheses||
hello
||comma||
moe's
tavern
||dash||
birthplace
of
the
rob
roy
||period||
||return||
moe_szyslak:
||left_parentheses||
into
phone
||rig

## Build the Neural Network
You'll build the components necessary to build a RNN by implementing the following functions below:
- get_inputs
- get_init_cell
- get_embed
- build_rnn
- build_nn
- get_batches

### Check the Version of TensorFlow and Access to GPU

In [8]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
from distutils.version import LooseVersion
import warnings
import tensorflow as tf

# Check TensorFlow Version
assert LooseVersion(tf.__version__) >= LooseVersion('1.0'), 'Please use TensorFlow version 1.0 or newer'
print('TensorFlow Version: {}'.format(tf.__version__))

# Check for a GPU
if not tf.test.gpu_device_name():
    warnings.warn('No GPU found. Please use a GPU to train your neural network.')
else:
    print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))

TensorFlow Version: 1.0.1


C:\Users\praveen.chaudhary\AppData\Local\conda\conda\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:14: UserWarning: No GPU found. Please use a GPU to train your neural network.
  


### Input
Implement the `get_inputs()` function to create TF Placeholders for the Neural Network.  It should create the following placeholders:
- Input text placeholder named "input" using the [TF Placeholder](https://www.tensorflow.org/api_docs/python/tf/placeholder) `name` parameter.
- Targets placeholder
- Learning Rate placeholder

Return the placeholders in the following the tuple `(Input, Targets, LearingRate)`

In [9]:
def get_inputs():
    """
    Create TF Placeholders for input, targets, and learning rate.
    :return: Tuple (input, targets, learning rate)
    """
    inputs = tf.placeholder(tf.int32,[None,None],name='input')
    targets = tf.placeholder(tf.int32,[None,None],name = 'targets')
    lr = tf.placeholder(tf.float32,name='learning_rate')
    
    return (inputs,targets,lr)


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_get_inputs(get_inputs)

Tests Passed


### Build RNN Cell and Initialize
Stack one or more [`BasicLSTMCells`](https://www.tensorflow.org/api_docs/python/tf/contrib/rnn/BasicLSTMCell) in a [`MultiRNNCell`](https://www.tensorflow.org/api_docs/python/tf/contrib/rnn/MultiRNNCell).
- The Rnn size should be set using `rnn_size`
- Initalize Cell State using the MultiRNNCell's [`zero_state()`](https://www.tensorflow.org/api_docs/python/tf/contrib/rnn/MultiRNNCell#zero_state) function
    - Apply the name "initial_state" to the initial state using [`tf.identity()`](https://www.tensorflow.org/api_docs/python/tf/identity)

Return the cell and initial state in the following tuple `(Cell, InitialState)`

In [10]:
def get_init_cell(batch_size, rnn_size):
    """
    Create an RNN Cell and initialize it.
    :param batch_size: Size of batches
    :param rnn_size: Size of RNNs
    :return: Tuple (cell, initialize state)
    """
    print(rnn_size)
    cell = tf.contrib.rnn.BasicLSTMCell(rnn_size)
    cell = tf.contrib.rnn.MultiRNNCell([cell]*2)
    initial_state = tf.identity(cell.zero_state(batch_size,tf.float32),name='initial_state')
    return (cell,initial_state)


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_get_init_cell(get_init_cell)

256
Tests Passed


### Word Embedding
Apply embedding to `input_data` using TensorFlow.  Return the embedded sequence.

In [11]:
def get_embed(input_data, vocab_size, embed_dim):
    """
    Create embedding for <input_data>.
    :param input_data: TF placeholder for text input.
    :param vocab_size: Number of words in vocabulary.
    :param embed_dim: Number of embedding dimensions
    :return: Embedded input.
    """
    
    embedding = tf.Variable(tf.truncated_normal((vocab_size,embed_dim),stddev=0.1))
    embed = tf.nn.embedding_lookup(embedding, input_data)
    return embed


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_get_embed(get_embed)

Tests Passed


### Build RNN
You created a RNN Cell in the `get_init_cell()` function.  Time to use the cell to create a RNN.
- Build the RNN using the [`tf.nn.dynamic_rnn()`](https://www.tensorflow.org/api_docs/python/tf/nn/dynamic_rnn)
 - Apply the name "final_state" to the final state using [`tf.identity()`](https://www.tensorflow.org/api_docs/python/tf/identity)

Return the outputs and final_state state in the following tuple `(Outputs, FinalState)` 

In [12]:
def build_rnn(cell, inputs):
    """
    Create a RNN using a RNN Cell
    :param cell: RNN Cell
    :param inputs: Input text data
    :return: Tuple (Outputs, Final State)
    """
    print(inputs)
    print(tf.transpose(inputs))
    Outputs, FinalState = tf.nn.dynamic_rnn(cell,inputs,initial_state=None,dtype=tf.float32)
    return (Outputs,tf.identity(FinalState,name='final_state'))


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_build_rnn(build_rnn)

Tensor("Placeholder:0", shape=(?, ?, 256), dtype=float32)
Tensor("transpose:0", shape=(256, ?, ?), dtype=float32)
Tests Passed


### Build the Neural Network
Apply the functions you implemented above to:
- Apply embedding to `input_data` using your `get_embed(input_data, vocab_size, embed_dim)` function.
- Build RNN using `cell` and your `build_rnn(cell, inputs)` function.
- Apply a fully connected layer with a linear activation and `vocab_size` as the number of outputs.

Return the logits and final state in the following tuple (Logits, FinalState) 

In [13]:
def build_nn(cell, rnn_size, input_data, vocab_size):
    embed = get_embed(input_data,vocab_size,300)
    print(embed)
    outputs,final_State = build_rnn(cell,embed)
    logits = tf.contrib.layers.fully_connected(outputs,num_outputs=vocab_size,activation_fn=tf.nn.relu,weights_initializer=tf.truncated_normal_initializer(mean=0.0,stddev=0.1),biases_initializer=tf.zeros_initializer())
    print(logits)
    return (logits,final_State)   


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_build_nn(build_nn)

Tensor("embedding_lookup:0", shape=(128, 5, 300), dtype=float32)
Tensor("embedding_lookup:0", shape=(128, 5, 300), dtype=float32)
Tensor("transpose:0", shape=(300, 5, 128), dtype=float32)
Tensor("fully_connected/Relu:0", shape=(128, 5, 27), dtype=float32)
Tests Passed


### Batches
Implement `get_batches` to create batches of input and targets using `int_text`.  The batches should be a Numpy array with the shape `(number of batches, 2, batch size, sequence length)`. Each batch contains two elements:
- The first element is a single batch of **input** with the shape `[batch size, sequence length]`
- The second element is a single batch of **targets** with the shape `[batch size, sequence length]`

If you can't fill the last batch with enough data, drop the last batch.

For exmple, `get_batches([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15], 2, 3)` would return a Numpy array of the following:
```
[
  # First Batch
  [
    # Batch of Input
    [[ 1  2  3], [ 7  8  9]],
    # Batch of targets
    [[ 2  3  4], [ 8  9 10]]
  ],
 
  # Second Batch
  [
    # Batch of Input
    [[ 4  5  6], [10 11 12]],
    # Batch of targets
    [[ 5  6  7], [11 12 13]]
  ]
]
```

In [14]:
def get_batches(int_text, batch_size, seq_length):
    """
    Return batches of input and target
    :param int_text: Text with the words replaced by their ids
    :param batch_size: The size of batch
    :param seq_length: The length of sequence
    :return: Batches as a Numpy array
    """
    from numpy.lib.stride_tricks import as_strided as strided
    n_batches = len(int_text)//(batch_size*seq_length)
    
    int_text = np.array(int_text)
    s=int_text.strides[0]
    print(s)
    
    batches=strided(int_text,shape=(n_batches,2,batch_size,seq_length),strides=(s * seq_length, s, s * seq_length*2, s))
    return batches

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_get_batches(get_batches)

4
Tests Passed


## Neural Network Training
### Hyperparameters
Tune the following parameters:

- Set `num_epochs` to the number of epochs.
- Set `batch_size` to the batch size.
- Set `rnn_size` to the size of the RNNs.
- Set `seq_length` to the length of sequence.
- Set `learning_rate` to the learning rate.
- Set `show_every_n_batches` to the number of batches the neural network should print progress.

In [15]:
# Number of Epochs
num_epochs = 82
# Batch Size
batch_size = 128
# RNN Size
rnn_size = 256
# Sequence Length
seq_length = 20
# Learning Rate
learning_rate = 0.01
# Show stats for every n number of batches
show_every_n_batches = 50

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
save_dir = './save'

### Build the Graph
Build the graph using the neural network you implemented.

In [16]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
from tensorflow.contrib import seq2seq

train_graph = tf.Graph()
with train_graph.as_default():
    vocab_size = len(int_to_vocab)
    input_text, targets, lr = get_inputs()
    input_data_shape = tf.shape(input_text)
    cell, initial_state = get_init_cell(input_data_shape[0], rnn_size)
    logits, final_state = build_nn(cell, rnn_size, input_text, vocab_size)

    # Probabilities for generating words
    probs = tf.nn.softmax(logits, name='probs')

    # Loss function
    cost = seq2seq.sequence_loss(
        logits,
        targets,
        tf.ones([input_data_shape[0], input_data_shape[1]]))

    # Optimizer
    optimizer = tf.train.AdamOptimizer(lr)

    # Gradient Clipping
    gradients = optimizer.compute_gradients(cost)
    capped_gradients = [(tf.clip_by_value(grad, -1., 1.), var) for grad, var in gradients]
    train_op = optimizer.apply_gradients(capped_gradients)

256
Tensor("embedding_lookup:0", shape=(?, ?, 300), dtype=float32)
Tensor("embedding_lookup:0", shape=(?, ?, 300), dtype=float32)
Tensor("transpose:0", shape=(300, ?, ?), dtype=float32)
Tensor("fully_connected/Relu:0", shape=(?, ?, 6780), dtype=float32)


## Train
Train the neural network on the preprocessed data.  If you have a hard time getting a good loss, check the [forms](https://discussions.udacity.com/) to see if anyone is having the same problem.

In [17]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
batches = get_batches(int_text, batch_size, seq_length)

with tf.Session(graph=train_graph) as sess:
    sess.run(tf.global_variables_initializer())

    for epoch_i in range(num_epochs):
        state = sess.run(initial_state, {input_text: batches[0][0]})

        for batch_i, (x, y) in enumerate(batches):
            feed = {
                input_text: x,
                targets: y,
                initial_state: state,
                lr: learning_rate}
            train_loss, state, _ = sess.run([cost, final_state, train_op], feed)

            # Show every <show_every_n_batches> batches
            if (epoch_i * len(batches) + batch_i) % show_every_n_batches == 0:
                print('Epoch {:>3} Batch {:>4}/{}   train_loss = {:.3f}'.format(
                    epoch_i,
                    batch_i,
                    len(batches),
                    train_loss))

    # Save Model
    saver = tf.train.Saver()
    saver.save(sess, save_dir)
    print('Model Trained and Saved')

4
Epoch   0 Batch    0/26   train_loss = 8.821
Epoch   1 Batch   24/26   train_loss = 5.415
Epoch   3 Batch   22/26   train_loss = 3.918
Epoch   5 Batch   20/26   train_loss = 2.596
Epoch   7 Batch   18/26   train_loss = 1.906
Epoch   9 Batch   16/26   train_loss = 1.720
Epoch  11 Batch   14/26   train_loss = 1.692
Epoch  13 Batch   12/26   train_loss = 1.673
Epoch  15 Batch   10/26   train_loss = 1.651
Epoch  17 Batch    8/26   train_loss = 1.644
Epoch  19 Batch    6/26   train_loss = 1.639
Epoch  21 Batch    4/26   train_loss = 1.635
Epoch  23 Batch    2/26   train_loss = 1.646
Epoch  25 Batch    0/26   train_loss = 1.652
Epoch  26 Batch   24/26   train_loss = 1.686
Epoch  28 Batch   22/26   train_loss = 1.671
Epoch  30 Batch   20/26   train_loss = 1.664
Epoch  32 Batch   18/26   train_loss = 1.670
Epoch  34 Batch   16/26   train_loss = 1.676
Epoch  36 Batch   14/26   train_loss = 1.677
Epoch  38 Batch   12/26   train_loss = 1.665
Epoch  40 Batch   10/26   train_loss = 1.647
Epoch  4

## Save Parameters
Save `seq_length` and `save_dir` for generating a new TV script.

In [18]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
# Save parameters for checkpoint
helper.save_params((seq_length, save_dir))

# Checkpoint

In [19]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import tensorflow as tf
import numpy as np
import helper
import problem_unittests as tests

_, vocab_to_int, int_to_vocab, token_dict = helper.load_preprocess()
seq_length, load_dir = helper.load_params()

## Implement Generate Functions
### Get Tensors
Get tensors from `loaded_graph` using the function [`get_tensor_by_name()`](https://www.tensorflow.org/api_docs/python/tf/Graph#get_tensor_by_name).  Get the tensors using the following names:
- "input:0"
- "initial_state:0"
- "final_state:0"
- "probs:0"

Return the tensors in the following tuple `(InputTensor, InitialStateTensor, FinalStateTensor, ProbsTensor)` 

In [20]:
def get_tensors(loaded_graph):
    """
    Get input, initial state, final state, and probabilities tensor from <loaded_graph>
    :param loaded_graph: TensorFlow graph loaded from file
    :return: Tuple (InputTensor, InitialStateTensor, FinalStateTensor, ProbsTensor)
    """
    InputTensor = loaded_graph.get_tensor_by_name('input:0')
    InitialStateTensor = loaded_graph.get_tensor_by_name('initial_state:0')
    FinalStateTensor = loaded_graph.get_tensor_by_name('final_state:0')
    ProbsTensor = loaded_graph.get_tensor_by_name('probs:0')
    # TODO: Implement Function
    return (InputTensor, InitialStateTensor, FinalStateTensor, ProbsTensor)


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_get_tensors(get_tensors)

Tests Passed


### Choose Word
Implement the `pick_word()` function to select the next word using `probabilities`.

In [21]:
def pick_word(probabilities, int_to_vocab):
    """
    Pick the next word in the generated text
    :param probabilities: Probabilites of the next word
    :param int_to_vocab: Dictionary of word ids as the keys and words as the values
    :return: String of the predicted word
    """
    index = np.argmax(probabilities)
    return int_to_vocab[1]


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_pick_word(pick_word)

Tests Passed


## Generate TV Script
This will generate the TV script for you.  Set `gen_length` to the length of TV script you want to generate.

In [22]:
gen_length = 200
# homer_simpson, moe_szyslak, or Barney_Gumble
prime_word = 'moe_szyslak'

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
loaded_graph = tf.Graph()
with tf.Session(graph=loaded_graph) as sess:
    # Load saved model
    loader = tf.train.import_meta_graph(load_dir + '.meta')
    loader.restore(sess, load_dir)

    # Get Tensors from loaded model
    input_text, initial_state, final_state, probs = get_tensors(loaded_graph)

    # Sentences generation setup
    gen_sentences = [prime_word + ':']
    prev_state = sess.run(initial_state, {input_text: np.array([[1]])})

    # Generate sentences
    for n in range(gen_length):
        # Dynamic Input
        dyn_input = [[vocab_to_int[word] for word in gen_sentences[-seq_length:]]]
        dyn_seq_length = len(dyn_input[0])

        # Get Prediction
        probabilities, prev_state = sess.run(
            [probs, final_state],
            {input_text: dyn_input, initial_state: prev_state})
        
        pred_word = pick_word(probabilities[dyn_seq_length-1], int_to_vocab)

        gen_sentences.append(pred_word)
    
    # Remove tokens
    tv_script = ' '.join(gen_sentences)
    for key, token in token_dict.items():
        ending = ' ' if key in ['\n', '(', '"'] else ''
        tv_script = tv_script.replace(' ' + token.lower(), key)
    tv_script = tv_script.replace('\n ', '\n')
    tv_script = tv_script.replace('( ', '(')
        
    print(tv_script)

moe_szyslak: dumbass dumbass dumbass dumbass dumbass dumbass dumbass dumbass dumbass dumbass dumbass dumbass dumbass dumbass dumbass dumbass dumbass dumbass dumbass dumbass dumbass dumbass dumbass dumbass dumbass dumbass dumbass dumbass dumbass dumbass dumbass dumbass dumbass dumbass dumbass dumbass dumbass dumbass dumbass dumbass dumbass dumbass dumbass dumbass dumbass dumbass dumbass dumbass dumbass dumbass dumbass dumbass dumbass dumbass dumbass dumbass dumbass dumbass dumbass dumbass dumbass dumbass dumbass dumbass dumbass dumbass dumbass dumbass dumbass dumbass dumbass dumbass dumbass dumbass dumbass dumbass dumbass dumbass dumbass dumbass dumbass dumbass dumbass dumbass dumbass dumbass dumbass dumbass dumbass dumbass dumbass dumbass dumbass dumbass dumbass dumbass dumbass dumbass dumbass dumbass dumbass dumbass dumbass dumbass dumbass dumbass dumbass dumbass dumbass dumbass dumbass dumbass dumbass dumbass dumbass dumbass dumbass dumbass dumbass dumbass dumbass dumbass dumbass dum

# The TV Script is Nonsensical
It's ok if the TV script doesn't make any sense.  We trained on less than a megabyte of text.  In order to get good results, you'll have to use a smaller vocabulary or get more data.  Luckly there's more data!  As we mentioned in the begging of this project, this is a subset of [another dataset](https://www.kaggle.com/wcukierski/the-simpsons-by-the-data).  We didn't have you train on all the data, because that would take too long.  However, you are free to train your neural network on all the data.  After you complete the project, of course.
# Submitting This Project
When submitting this project, make sure to run all the cells before saving the notebook. Save the notebook file as "dlnd_tv_script_generation.ipynb" and save it as a HTML file under "File" -> "Download as". Include the "helper.py" and "problem_unittests.py" files in your submission.